<a href="https://colab.research.google.com/github/0xc3nt4ur10n/TorrentToGDrive/blob/main/TorrentToGDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='0f81c2'>__Torrent to Google Drive Downloader__

### <font color='0f81c1'>Install Dependency

In [ ]:
#@markdown <h3>Step #1. Install libtorrent and Initialize Session

# 🛠️ Install packages
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent -y

import libtorrent as lt

# Start libtorrent session
ses = lt.session()
ses.listen_on(6881, 6891)

print("\n🚀 Libtorrent session started!")

### <font color='0f81c1'>Mount Google Drive

In [ ]:
#@markdown <h3>Step #2. Mount Google Drive

from google.colab import drive, files

# Mount Google Drive to save torrents
drive.mount("/content/drive")

## <font color='0f81c1'>Add Torrent Source



In [ ]:
#@markdown <h3>Step #3. Add From Magnet Link or Torrent File

import os

# Create save path
save_path = '/content/drive/MyDrive/Torrents/Movies'
os.makedirs(save_path, exist_ok=True)

# Initialize list of downloads
downloads = []

# Add torrents from multiple magnet links or .torrent files
while True:
    source = input("🔗 Enter a magnet link or type 'file' to upload a .torrent file or 'exit' to stop: ")

    if source.lower() == 'exit':
        print("\n👋 Exiting input mode.")
        break

    elif source.lower() == 'file':
        print("📁 Uploading a .torrent file...")
        uploaded = files.upload()
        if uploaded:
            torrent_file = list(uploaded.keys())[0]
            try:
                info = lt.torrent_info(torrent_file)
                h = ses.add_torrent({'ti': info, 'save_path': save_path})
                downloads.append(h)
                print(f"\n🎬 Starting: {h.status().name}")
            except Exception as e:
                print(f"⚠️ Error loading torrent: {e}")
        else:
            print("⚠️ No file uploaded.")

    elif source.startswith("magnet:?xt="):
        print("\n🔗 Adding magnet link...")
        try:
            params = lt.parse_magnet_uri(source)
            params.save_path = save_path
            h = ses.add_torrent(params)
            downloads.append(h)
            print(f"\n🎬 Starting: {h.status().name}")
        except Exception as e:
            print(f"⚠️ Invalid magnet link: {e}")
    else:
        print("❌ Invalid input. Try again.")

## <font color='0f81c1'>Start Download

In [ ]:
#@markdown <h3>Step #4. Start Download

import time
from IPython.display import clear_output

# Lists
ongoing_downloads = downloads
completed_downloads = []

# ⏳ Monitor downloads
while ongoing_downloads:
    clear_output(wait=True)
    next_iteration_downloads = []
    display_list = []

    # Collect display strings and organize
    for download in ongoing_downloads:
        s = download.status()

        # 🌀 State strings
        state_str = [
            'queued',
            'checking',
            'downloading metadata',
            'downloading',
            'finished',
            'seeding',
            'allocating',
            'checking fastresume'
        ]
        state = state_str[s.state] if s.state < len(state_str) else "unknown"

        # ETA calculation with hours
        if s.download_rate > 0 and s.total_wanted > 0:
            eta_seconds = (s.total_wanted - s.total_done) / s.download_rate
            hours = int(eta_seconds) // 3600  # Calculate hours
            minutes = (int(eta_seconds) % 3600) // 60  # Calculate minutes
            seconds = int(eta_seconds) % 60  # Calculate remaining seconds

            if hours > 0:
                eta_str = f"ETA: {hours}h {minutes}m {seconds}s"
            elif minutes > 0:
                eta_str = f"ETA: {minutes}m {seconds}s"
            else:
                eta_str = f"ETA: {seconds}s"
        else:
            eta_str = "ETA: ∞"

        if s.is_seeding:
            display_list.insert(0, f'🎬 {s.name} ✅ {s.progress * 100:.2f}% complete! 🌀 seeding ⬇️ 0.0 kB/s ⬆️ {s.upload_rate / 1000:.1f} kB/s 👥 peers: {s.num_peers}')
            completed_downloads.append(download)
        else:
            display_list.append(f'🎬 {s.name} ⬇️ {s.progress * 100:.2f}% ⏳ {eta_str} 🌀 {state} ⬇️ {s.download_rate / 1000:.1f} kB/s ⬆️ {s.upload_rate / 1000:.1f} kB/s 👥 peers: {s.num_peers}')
            next_iteration_downloads.append(download)

    # Display the download status in sorted order
    for line in display_list:
        print(line)

    # Update ongoing downloads
    ongoing_downloads = next_iteration_downloads
    time.sleep(1)